In [1]:
#READ DATA
using CSV, DataFrames, JuMP, Gurobi

df_nut = CSV.read("data_nutrients.csv", DataFrame, delim = ";")
df_rec = CSV.read("data_recipes.csv", DataFrame, delim = ";")
df_price = CSV.read("price_ing.csv", DataFrame, delim = ";")
df_recConst = CSV.read("recipes_constr_cleaned.csv", DataFrame)

,Name,Servings,OrigIngredient
,String,Int64,String
1,5 A Day Salad,4,4 cups spinach (fresh)
2,5 A Day Salad,4,4 cups romaine lettuce
3,5 A Day Salad,4,"2 cups green pepper (chopped, or use red, yellow, or orange)"
4,5 A Day Salad,4,2 cups cherry tomatoes
5,5 A Day Salad,4,1 cup broccoli (chopped)
6,5 A Day Salad,4,1 cup cauliflower (chopped)
7,5 A Day Salad,4,1 cup yellow squash (sliced)
8,5 A Day Salad,4,1 cup cucumber (sliced)
9,5 A Day Salad,4,2 cups carrot (chopped)


In [2]:
# Create dictionary that maps recipe names to lists of ingredients
mydict = Dict(
    Pair.(
        transform(combine(groupby(df_recConst, :Name), :Match => Set)).Name,
        transform(combine(groupby(df_recConst, :Name), :Match => Set)).Match_Set
    )
)

Dict{String, Set{String}} with 95 entries:
  "Pineapple Chicken"       => Set(["Jalapeno Peppers", "Purified Water", "Pine…
  "New England Johnny Cake" => Set(["Planet Oat Original Oat Milk", "Large Brow…
  "Butternut Squash with B… => Set(["Goya Black Beans", "Badia Ground Oregano",…
  "Garlic Ginger Ramen wit… => Set(["Best Yet Frozen Stir Fry Vegetables", "Pur…
  "Granola Bars"            => Set(["Daily Table Raisins", "Madame Gougousse Co…
  "Skillet Pasta Dinner"    => Set(["Best Yet Tomato Sauce", "Purified Water", …
  "Avocado and Corn Salsa"  => Set(["Limes", "Avocados", "Cilantro Bunch", "Gra…
  "Breakfast Burrito with … => Set(["Hood Whole Milk", "Large Brown Eggs", "Bee…
  "Fruity Homemade Oatmeal" => Set(["Purified Water", "Best Yet Old Fashioned O…
  "Old Fashioned Bread Pud… => Set(["White Onions", "Planet Oat Original Oat Mi…
  "Ivory Coast Bananas wit… => Set(["Purified Water", "Bananas", "Cloverdale Sa…
  "Fiesta Lettuce Wraps an… => Set(["Hood All Natural Sour Cream",

In [3]:
# Create dictionary that maps meal number (1 = breakfast, 2 = lunch, 3 = dinner) to possible recipes
stacked = subset(
    stack(
        unique(select(df_recConst, :Name, :B, :L, :D, :S)), 
        2:5
    ),
    :value => ByRow(value -> value),
    skipmissing=true
)

temp_dict = Dict(
    Pair.(
        transform(combine(groupby(stacked, :variable), :Name => Set)).variable,
        transform(combine(groupby(stacked, :variable), :Name => Set)).Name_Set
    )
)

mydict2 = Dict(
    1 => temp_dict["B"],
    2 => temp_dict["L"],
    3 => temp_dict["D"],
    # 4 = > temp_dict["S"] # Uncomment to include snack/dessert items
)

Dict{Int64, Set{String}} with 3 entries:
  2 => Set(["Pineapple Chicken", "Garden Cannellini Bean Salad", "Veggie Bean W…
  3 => Set(["Pineapple Chicken", "Garden Cannellini Bean Salad", "Veggie Bean W…
  1 => Set(["Cinnamon Raisin Almond Balls", "Fruit Salad with Yogurt", "Flour T…

In [4]:
#DEFINE SETS AND INDICES

#----------------------------------------------------------------------
#set of days (d)
d = 7
D = [j for j in range(1,d)]

#set of meals (m)
m = 3
M = [j for j in range(1,m)]

#set of nutrients (n)
N = unique(df_nut, "nutrients").nutrients

#set of nutrients relaxed daily lower bound
lownut = copy(N)
deleteat!(lownut, findall(x->x=="Calories",lownut))
deleteat!(lownut, findall(x->x=="Fat",lownut))
deleteat!(lownut, findall(x->x=="Protein",lownut))
deleteat!(lownut, findall(x->x=="Carb",lownut))

#set of nutrients with daily upper bound
uppnut = ["Calories"]

#set of nutrients with per meal lower and upper bound
uppnutmeal = ["Calories"]

#set of recipes (r)
R = unique(df_recConst[:,1])

#set of people (p)
P = unique(df_nut, "person").person

#set of ingredients (i)
I = unique(df_recConst, "Match").Match

#----------------------------------------------------------------------
#indice rmd
rmd = [(l,j,k) for l in R for j in M for k in D if l in mydict2[j]]

#indice rmdp
rmdp = [(l,j,k,f) for l in R for j in M for k in D for f in P if l in mydict2[j]]

#indice np
np = [(u,f) for u in N for f in P]

#indice nr
nr =[(u,l) for u in N for l in R]

#indice ir
ir = [(row.Match,row.Name) for row in eachrow(df_recConst)]

757-element Vector{Tuple{String, String}}:
 ("Baby Spinach", "5 A Day Salad")
 ("Romaine Lettuce Hearts", "5 A Day Salad")
 ("Green Bell Peppers", "5 A Day Salad")
 ("Beefsteak Tomatoes", "5 A Day Salad")
 ("Broccoli", "5 A Day Salad")
 ("Cauliflower", "5 A Day Salad")
 ("Butternut Squash", "5 A Day Salad")
 ("Cucumber", "5 A Day Salad")
 ("Carrots", "5 A Day Salad")
 ("Zucchini Squash", "5 A Day Salad")
 ("Pork Chops", "Apple Carrot Soup")
 ("Fuji Apples", "Apple Carrot Soup")
 ("Carrots", "Apple Carrot Soup")
 ⋮
 ("Large Brown Eggs", "Zucchini Bread")
 ("Best Yet Granulated Sugar", "Zucchini Bread")
 ("Best Yet Vegetable Oil", "Zucchini Bread")
 ("Zucchini Squash", "Zucchini Bread")
 ("Equal Exchange French Vanilla", "Zucchini Bread")
 ("Best Yet All Purpose Flour", "Zucchini Bread")
 ("Heckers All Natural 100 Whole Wheat Flour", "Zucchini Bread")
 ("Badia Garlic Salt", "Zucchini Bread")
 ("Best Yet Baking Soda", "Zucchini Bread")
 ("Badia Cinnamon Powder", "Zucchini Bread")
 ("Best 

In [5]:
#DEFINE DATA

#daily needs of nutrient n by person p
need = Dict((j,k) => df_nut[(df_nut.nutrients .== j) .& (df_nut.person .== k), :lowerbound][1] for (j,k) in np)

#daily max of nutrient n by person p
####max = Dict((j,k) => df_nut[(df_nut.nutrients .== j) .& (df_nut.person .== k), :upperbound][1] for (j,k) in np)
max = Dict((j,k) => df_nut[(df_nut.nutrients .== j) .& (df_nut.person .== k), :upperbound][1] for j in uppnut for k in P)

#price of ingredient i per unit of measurement of the ingredient
price = Dict(j => df_price[(df_price.ingredient .== j), :price][1] for j in I)

#quantity of nutrient n granted by one portion of recipe r
nutrec = Dict((j,k) => df_rec[(df_rec.recipe .== k), j][1] for (j,k) in nr)

#quantity of ingredient i needed by one portion of recipe r
recing = Dict((j,k) => df_recConst[(df_recConst.Name .== k) .& (df_recConst.Match .== j), :SI_Qty_Per_Serving][1]
    for (j,k) in ir)

#quantity of calories per unit of macros
qtycal = Dict("Protein" => 4, "Carb" => 4, "Fat" => 9)

#minimum contribution from each macro to calories
ratio = Dict("Protein" => 0.10, "Carb" => 0.45, "Fat" => 0.20)

#maximum contribution from each macro to calories
ratio2 = Dict("Protein" => 0.35, "Carb" => 0.65, "Fat" => 0.35)

#minimum percentage of calories in each meal
perc = [0.10,0.35,0.30]

#maximum percentage of calories in each meal
perc2 = [0.25,0.55,0.50]

#maximum percentage of meals for which the same recipe can be chosen
samerecperc = 0.1

#for (j,k) in nr
    #println(j,", ",k,": ",nutrec[(j,k)])
#end

0.1

In [6]:
#DEFINE AND SHOW MODEL

#--model
mdl = Model(with_optimizer(Gurobi.Optimizer))

#--decision variables
@variables mdl begin
    X[rmd], Bin
    Q[rmdp]>=0
    Y[I]>=0 #need to understand if it is possible to define some of these as integer and others as continuous
end

#change variable Y to be integer for certain ingredients
for s in I
    if df_price[(df_price.ingredient .== s), :unit][1] == "each" || df_price[(df_price.ingredient .== s), :unit][1] == "ct"
        set_integer(Y[s])
    end
end

#--objective function
@objective(mdl, Min, 0.99999*sum(Y[j]*price[j] for j in I)
    - 0.00001*sum((sum(Q[(h,v,s,a)]*nutrec[(j,h)] for (h,v,s,a) in rmdp if s==i && a==l) - need[(j,l)]) for i in D, 
            j in lownut, l in P))

#--constraints
@constraints mdl begin
    constraint_1[i in M,j in D],
    sum(X[(l,v,s)] for (l,v,s) in rmd if v==i && s==j) == 1
    
    constraint_2[i in D,j in ["Calories"],l in P],
    sum(Q[(h,v,s,a)]*nutrec[(j,h)] for (h,v,s,a) in rmdp if s==i && a==l) >= need[(j,l)] 

    constraint_3[(i,j,l) in rmd],
    sum(Q[(i,j,l,k)] for k in P) <= 1000*X[(i,j,l)]
    
    constraint_4[s in I],
    sum(Q[(i,j,l,k)]*recing[(s,i)] for (i,j,l,k) in rmdp if s in mydict[i]) <= Y[s]
    
    constraint_5[i in D,j in uppnut,l in P],
    sum(Q[(h,v,s,a)]*nutrec[(j,h)] for (h,v,s,a) in rmdp if s==i && a==l) <= max[(j,l)]
    
    constraint_6[k in M,i in D,j in uppnutmeal,l in P],
    (sum(Q[(h,v,s,a)]*nutrec[(j,h)] for (h,v,s,a) in rmdp if s==i && a==l && v==k)
       >= perc[k]*sum(Q[(h,v,s,a)]*nutrec[(j,h)] for (h,v,s,a) in rmdp if s==i && a==l))
    
    constraint_7[k in M,i in D,j in uppnutmeal,l in P],
    (sum(Q[(h,v,s,a)]*nutrec[(j,h)] for (h,v,s,a) in rmdp if s==i && a==l && v==k)
       <= perc2[k]*sum(Q[(h,v,s,a)]*nutrec[(j,h)] for (h,v,s,a) in rmdp if s==i && a==l))
    
    constraint_8[i in mydict2[2]],
    sum(X[(h,v,s)] for (h,v,s) in rmd if h==i) - samerecperc*(d*(m-1)) <= 0 #(v in [2,3] &&)
    
    constraint_9[i in mydict2[2], j in D],
    sum(X[(h,v,s)] for (h,v,s) in rmd if h==i && s==j) <= 1
    
    constraint_10[i in ["Protein","Carb","Fat"],j in ["Calories"], k in D, l in P],
    sum(Q[(h,v,s,a)]*nutrec[(i,h)]*qtycal[i] for (h,v,s,a) in rmdp if s==k && a==l) >= ratio[i]*sum(Q[(h,v,s,a)]*nutrec[(j,h)] for (h,v,s,a) in rmdp if s==k && a==l)
    
    constraint_11[i in ["Protein","Carb","Fat"],j in ["Calories"], k in D, l in P],
    sum(Q[(h,v,s,a)]*nutrec[(i,h)]*qtycal[i] for (h,v,s,a) in rmdp if s==k && a==l) <= ratio2[i]*sum(Q[(h,v,s,a)]*nutrec[(j,h)] for (h,v,s,a) in rmdp if s==k && a==l)
    
    constraint_12[i in D,j in lownut,l in P],
    sum(Q[(h,v,s,a)]*nutrec[(j,h)] for (h,v,s,a) in rmdp if s==i && a==l) >= 0.1
end

Set parameter Username
Academic license - for non-commercial use only - expires 2023-03-27


(2-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},2,...} with index sets:
    Dimension 1, [1, 2, 3]
    Dimension 2, [1, 2, 3, 4, 5, 6, 7]
And data, a 3×7 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 constraint_1[1,1] : X[("Apple Chunk Cake", 1, 1)] + X[("Apple Oatmeal Muffins", 1, 1)] + X[("Apple Sandwiches", 1, 1)] + X[("Apple Slice Pancakes", 1, 1)] + X[("Apple-Stuffed Squash", 1, 1)] + X[("Avocado and Corn Salsa", 1, 1)] + X[("Banana Bread", 1, 1)] + X[("Banana Cupcakes", 1, 1)] + X[("Banana Oatmeal Raisin Cookies", 1, 1)] + X[("Banana Waldorf", 1, 1)] + X[("Blueberry Muffins", 1, 1)] + X[("Breakfast Burrito with Salsa", 1, 1)] + X[("Cinnamon Raisin Almond Balls", 1, 1)] + X[("Cinnamon Vanilla Granola", 1, 1)] + X[("Flour Tortillas", 1, 1

In [7]:
#--solve the model
optimize!(mdl)

choosen_recipes = value.(X)
quantities = value.(Q)
needed_ing = value.(Y)

result = objective_value(mdl)

println("Choosen recipes: ", choosen_recipes)
println("Quantities of recipes: ", quantities)
println("Needed_ingredients: ", needed_ing)
println("Cost: ", result)

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[arm])
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads
Optimize a model with 2584 rows, 4511 columns and 155865 nonzeros
Model fingerprint: 0x04f08d1d
Variable types: 3400 continuous, 1111 integer (1092 binary)
Coefficient statistics:
  Matrix range     [3e-02, 2e+03]
  Objective range  [9e-04, 4e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-01, 2e+03]
Presolve removed 595 rows and 42 columns
Presolve time: 0.15s
Presolved: 1989 rows, 4469 columns, 106784 nonzeros
Variable types: 3365 continuous, 1104 integer (1085 binary)

Root relaxation: objective 1.423001e+01, 1278 iterations, 0.03 seconds (0.06 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   14.23001    0   47          -   14.23001      -     -    0s
     0     0   14.23001    0   91          -   14.2300

Choosen recipes: 1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, [("5 A Day Salad", 2, 1), ("5 A Day Salad", 2, 2), ("5 A Day Salad", 2, 3), ("5 A Day Salad", 2, 4), ("5 A Day Salad", 2, 5), ("5 A Day Salad", 2, 6), ("5 A Day Salad", 2, 7), ("5 A Day Salad", 3, 1), ("5 A Day Salad", 3, 2), ("5 A Day Salad", 3, 3)  …  ("Whole Wheat Muffins", 1, 5), ("Whole Wheat Muffins", 1, 6), ("Whole Wheat Muffins", 1, 7), ("Zucchini Bread", 1, 1), ("Zucchini Bread", 1, 2), ("Zucchini Bread", 1, 3), ("Zucchini Bread", 1, 4), ("Zucchini Bread", 1, 5), ("Zucchini Bread", 1, 6), ("Zucchini Bread", 1, 7)]
And data, a 1092-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 1.0
 1.0
 1.0
 0.0
 0.0
 0.0
 0.0

 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0


 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 2.0826736331011184
 2.366674583069453
 2.0826736331011184
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 

In [8]:
println("Serving size:")
for (h,v,s,a) in rmdp
    if quantities[(h,v,s,a)] != 0
        println(h,",",v,",",s,",",a,": ",quantities[(h,v,s,a)])
    end
end

println("\nCalories per meal:")
for (h,v,s,a) in rmdp
    if quantities[(h,v,s,a)] != 0
        println(h,",",v,",",s,",",a,": ",quantities[(h,v,s,a)]*nutrec[("Calories",h)])
    end
end


println("\nCalories:")
for x in D
    for y in P
        println(y,",",x,": ",sum(quantities[(h,v,s,a)]*nutrec[("Calories",h)] for (h,v,s,a) in rmdp if x==s && a==y))
    end
end

println("\nProteins:")
for x in D
    for y in P
        println(y,",",x,": ",sum(quantities[(h,v,s,a)]*nutrec[("Protein",h)] for (h,v,s,a) in rmdp if x==s && a==y))
    end
end

println("\nCarbs:")
for x in D
    for y in P
        println(y,",",x,": ",sum(quantities[(h,v,s,a)]*nutrec[("Carb",h)] for (h,v,s,a) in rmdp if x==s && a==y))
    end
end

println("\nFat:")
for x in D
    for y in P
        println(y,",",x,": ",sum(quantities[(h,v,s,a)]*nutrec[("Fat",h)] for (h,v,s,a) in rmdp if x==s && a==y))
    end
end

println("\nVitD:")
for x in D
    for y in P
        println(y,",",x,": ",sum(quantities[(h,v,s,a)]*nutrec[("VitaminD",h)] for (h,v,s,a) in rmdp if x==s && a==y))
    end
end

println("\nVitA:")
for x in D
    for y in P
        println(y,",",x,": ",sum(quantities[(h,v,s,a)]*nutrec[("VitaminA",h)] for (h,v,s,a) in rmdp if x==s && a==y))
    end
end

Serving size:
Apple-Stuffed Squash,1,5,1: 3.173076923076923
Apple-Stuffed Squash,1,5,2: 3.605769230769231
Apple-Stuffed Squash,1,5,3: 3.173076923076923
Apple-Stuffed Squash,1,6,1: 3.173076923076923
Apple-Stuffed Squash,1,6,2: 3.605769230769231
Apple-Stuffed Squash,1,6,3: 3.173076923076923
Apple-Stuffed Squash,1,7,1: 3.173076923076923
Apple-Stuffed Squash,1,7,2: 3.605769230769231
Apple-Stuffed Squash,1,7,3: 3.173076923076923
Baked Pork Chops,2,7,1: 3.119707127734651
Baked Pork Chops,2,7,2: 3.545121736062103
Baked Pork Chops,2,7,3: 3.119707127734651
Banana Cupcakes,1,4,1: 1.9549763033175356
Banana Cupcakes,1,4,2: 2.221563981042654
Banana Cupcakes,1,4,3: 1.9549763033175356
Black Beans,2,2,1: 2.578125
Black Beans,2,2,2: 2.9296875
Black Beans,2,2,3: 2.578125
Easy Oven Packet Caribbean Tilapia with Pears and Carnival Roasted Potatoes,3,3,1: 1.475406177468309
Easy Oven Packet Caribbean Tilapia with Pears and Carnival Roasted Potatoes,3,3,2: 1.6765979289412605
Easy Oven Packet Caribbean Tilapi